In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import sys
sys.path.insert(0,'drive/Fast-Pytorch/Learning_Pytorch')
!ls drive

In [0]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [0]:
CUDA = torch.cuda.is_available()
print("CUDA:",CUDA)
device = torch.device("cuda" if CUDA else "cpu")

CUDA: True


In [0]:
lines_filepath=os.path.join("drive/Fast-Pytorch/Learning_Pytorch/datasets/cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath=os.path.join("drive/Fast-Pytorch/Learning_Pytorch/datasets/cornell movie-dialogs corpus", "movie_conversations.txt")

corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("drive/Fast-Pytorch/Learning_Pytorch/datasets/cornell movie-dialogs corpus", corpus_name)


In [0]:
with open(lines_filepath, 'r', encoding="utf8", errors='ignore') as file:
  lines=file.readlines()
for line in lines[:8]:
  print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [0]:
line_fields=["lineID", "characterID", "movieID", "character", "text"]
lines={}
with open(lines_filepath, 'r', encoding="iso-8859-1", errors='ignore') as file:
  for line in file:
    values = line.split(" +++$+++ ")
    lineObj={}
    for i, field in enumerate(line_fields):
      lineObj[field]= values[i]
    lines[lineObj['lineID']] = lineObj

In [0]:
lines

In [0]:
conv_fields=["characterID", "character2ID", "movieID", "utteranceIDs"]
conversations=[]
with open(conv_filepath, 'r', encoding="iso-8859-1", errors='ignore') as file:
  for line in file:
    values = line.split(" +++$+++ ")
    convObj={}
    for i, field in enumerate(conv_fields):
      convObj[field]= values[i]
    lineIds = eval(convObj["utteranceIDs"])
    convObj["lines"] = []
    for lineId in lineIds:
      convObj["lines"].append(lines[lineId])
    conversations.append(convObj)

In [0]:
conversations[0]

{'character2ID': 'u2',
 'characterID': 'u0',
 'lines': [{'character': 'BIANCA',
   'characterID': 'u0',
   'lineID': 'L194',
   'movieID': 'm0',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'character': 'CAMERON',
   'characterID': 'u2',
   'lineID': 'L195',
   'movieID': 'm0',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'character': 'BIANCA',
   'characterID': 'u0',
   'lineID': 'L196',
   'movieID': 'm0',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'character': 'CAMERON',
   'characterID': 'u2',
   'lineID': 'L197',
   'movieID': 'm0',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}],
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n"}

In [0]:
qa_pairs=[]
for conversation in conversations:
  for i in range(len(conversation["lines"])-1):
    inputLine=conversation["lines"][i]["text"].strip()
    targetLine=conversation["lines"][i+1]["text"].strip()
    if inputLine and targetLine:
      qa_pairs.append([inputLine,targetLine])

In [0]:
datafile=os.path.join("drive/Fast-Pytorch/Learning_Pytorch/datasets/cornell movie-dialogs corpus", "formatted_movie_lines.txt")
delimiter='\t'
delimiter= str(codecs.decode(delimiter, "unicode_escape"))

print("Writing new formatted file ")
with open(datafile, 'w', encoding='utf-8') as file:
  writer = csv.writer(file, delimiter=delimiter)
  for pair in qa_pairs:
    writer.writerow(pair)
print("Done")

Writing new formatted file 
Done


In [0]:
datafile=os.path.join("drive/Fast-Pytorch/Learning_Pytorch/datasets/cornell movie-dialogs corpus", "formatted_movie_lines.txt")
with open(datafile, 'rb') as file:
  lines=file.readlines()
for line in lines[:8]:
  print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\n"
b'Why?\tUnsolved myster

In [0]:
PAD_token =0
SOS_token =1
EOS_token =2

class Vocabulary:
  def __init__(self,name):
    self.name=name
    self.word2index={}
    self.word2count={}
    self.index2word={PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
    self.num_words=3
    
  def addSentence(self,sentence):
    for word in sentence.split(' '):
      self.addWord(word)
  
  def addWord(self,word):
    if word not in self.word2index:
      self.word2index[word] = self.num_words
      self.word2count[word]=1
      self.index2word[self.num_words]=word
      self.num_words+=1
    else:
      self.word2count[word]+=1
      
  def trim(self,min_count):
    keep_words=[]
    for k, v in self.word2count.items():
      if v>=min_count:
        keep_words.append(k)
        
      print('keep_words {} / {} = {:.4f}'.format( len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index) ))
      
      self.word2index ={}
      self.word2count={}
      self.index2word={PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
      self.num_words=3
      
      for word in keep_words:
        self.addWord(word)

In [0]:
def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [0]:
def normalizeString(s):
  s=unicodeToAscii(s.lower().strip())
  s=re.sub(r"([.!?])", r" \1", s)
  s=re.sub(r"[^a-zA-Z.!?]+", r" ", s)
  s=re.sub(r"\s+", r" ",s).strip()
  return s

In [0]:
normalizeString("aa123aa!s's   dd?")

'aa aa !s s dd ?'

In [0]:
datafile=os.path.join("drive/Fast-Pytorch/Learning_Pytorch/datasets/cornell movie-dialogs corpus", "formatted_movie_lines.txt")

print("Reading formatted file ")
lines= open(datafile, encoding='utf-8').read().strip().split('\n')
pairs=[[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print("Done reading")
voc = Vocabulary("cornell movie-dialogs corpus")

Reading formatted file 
Done reading


In [0]:
MAX_LENGTH=10

def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Vocabulary(corpus_name)
    return voc, pairs

def filterPair(p):
   return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH
  
def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [0]:
pairs=[pair for pair in pairs if len(pair)>1]
print("There are {} pairs/conservations in dataset".format(len(pairs)))
pairs= filterPairs(pairs)
print("After filtering, There are {} pairs/conservations in dataset".format(len(pairs)))

There are 64271 pairs/conservations in dataset
After filtering, There are 64271 pairs/conservations in dataset


In [0]:
for pair in pairs:
  voc.addSentence(pair[0])
  voc.addSentence(pair[1])
print("Counted words:", voc.num_words)
for pair in pairs[:10]:
  print(pair)

Counted words: 18008
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [0]:
MIN_COUNT=3

def trimRareWords(voc, pairs, MIN_COUNT):
  voc.trim(MIN_COUNT)
  keep_pairs=[]
  for pair in pairs:
    input_sentence=pair[0]
    output_sentence=pair[1]
    keep_input=True
    keep_output=True
    
    for word in input_sentence.split(' '):
      if word not in voc.word2index:
        keep_input=False
        break
    for word in output_sentence.split(' '):
      if word not in voc.word2index:
        keep_output=False
        break
    
    if keep_input and keep_output:
      keep_pairs.append(pair)
      
  print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs),len(keep_pairs), len(keep_pairs)/len(pairs)))
  return keep_pairs

pairs= trimRareWords(voc, pairs, MIN_COUNT)

keep_words 1 / 18005 = 0.0001
keep_words 2 / 1 = 2.0000
keep_words 3 / 2 = 1.5000
keep_words 4 / 3 = 1.3333
keep_words 5 / 4 = 1.2500
keep_words 6 / 5 = 1.2000
keep_words 7 / 6 = 1.1667
keep_words 8 / 7 = 1.1429
keep_words 9 / 8 = 1.1250
keep_words 10 / 9 = 1.1111
keep_words 11 / 10 = 1.1000
keep_words 12 / 11 = 1.0909
keep_words 13 / 12 = 1.0833
keep_words 14 / 13 = 1.0769
keep_words 15 / 14 = 1.0714
keep_words 16 / 15 = 1.0667
keep_words 17 / 16 = 1.0625
keep_words 18 / 17 = 1.0588
keep_words 19 / 18 = 1.0556
keep_words 20 / 19 = 1.0526
keep_words 21 / 20 = 1.0500
keep_words 22 / 21 = 1.0476
keep_words 23 / 22 = 1.0455
keep_words 24 / 23 = 1.0435
keep_words 25 / 24 = 1.0417
keep_words 26 / 25 = 1.0400
keep_words 27 / 26 = 1.0385
keep_words 28 / 27 = 1.0370
keep_words 29 / 28 = 1.0357
keep_words 29 / 29 = 1.0000
keep_words 30 / 29 = 1.0345
keep_words 31 / 30 = 1.0333
keep_words 32 / 31 = 1.0323
keep_words 33 / 32 = 1.0312
keep_words 34 / 33 = 1.0303
keep_words 35 / 34 = 1.0294
keep_wo

In [0]:
def indexesFromSentence(voc,sentence):
  return [voc.word2index[word] for word in sentence.split(' ')]+[EOS_token]

In [0]:
pairs[1][0]

'you have my word . as a gentleman'

In [0]:
indexesFromSentence(voc,pairs[1][0])

[7, 8, 9, 10, 4, 11, 12, 13, 2]

In [0]:
inp=[]
out=[]
for pair in pairs[:10]:
  inp.append(pair[0])
  out.append(pair[1])
print(inp)
print(len(inp))
indexes=[indexesFromSentence(voc,sentence) for sentence in inp]
indexes

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'wow']
10


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 32, 22, 6, 2],
 [34, 35, 4, 4, 4, 2],
 [36, 37, 38, 39, 7, 40, 41, 42, 4, 2],
 [43, 2],
 [48, 7, 49, 41, 46, 50, 6, 2],
 [51, 52, 53, 6, 2],
 [59, 2]]

In [0]:
def zeroPadding(l, fillvalue=0):
  return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [0]:
leng=[len(ind) for ind in indexes]
max(leng)

10

In [0]:
test_result=zeroPadding(indexes)
print(len(test_result))
test_result

10


[(3, 7, 16, 8, 34, 36, 43, 48, 51, 59),
 (4, 8, 4, 32, 35, 37, 2, 7, 52, 2),
 (2, 9, 2, 22, 4, 38, 0, 49, 53, 0),
 (0, 10, 0, 6, 4, 39, 0, 41, 6, 0),
 (0, 4, 0, 2, 4, 7, 0, 46, 2, 0),
 (0, 11, 0, 0, 2, 40, 0, 50, 0, 0),
 (0, 12, 0, 0, 0, 41, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 42, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [0]:
def binaryMatrix(l, value=0):
  m=[]
  for i, seq in enumerate(l):
    m.append([])
    for token in seq:
      if token == PAD_token:
        m[i].append(0)
      else:
        m[i].append(1)
  return m

In [0]:
binary_result =binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [0]:
def inputVar(l, voc):
  indexes_batch= [indexesFromSentence(voc, sentence) for sentence in l]
  lengths= torch.tensor([len(indexes) for indexes in indexes_batch])
  padList= zeroPadding(indexes_batch)
  padVar= torch.LongTensor(padList)
  return padVar, lengths

In [0]:
def outputVar(l, voc):
  indexes_batch= [indexesFromSentence(voc, sentence) for sentence in l]
  max_target_len= max([len(indexes) for indexes in indexes_batch])
  padList= zeroPadding(indexes_batch)
  mask= binaryMatrix(padList)
  mask= torch.ByteTensor(mask)
  padVar= torch.LongTensor(padList)
  return padVar, mask, max_target_len

In [0]:
def batch2TrainData(voc, pair_batch):
  pair_batch.sort(key=lambda x:len(x[0].split(" ")), reverse=True)
  input_batch, output_batch = [], []
  for pair in pair_batch:
    input_batch.append(pair[0])
    output_batch.append(pair[1])
  inp, lengths=inputVar(input_batch, voc)
  output, mask, max_target_len = outputVar(output_batch, voc)
  return inp, lengths, output, mask, max_target_len

In [0]:
small_batch_size=5
batches=batch2TrainData(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:")
print(input_variable)
print("lengths:", lengths)
print("target_variable:")
print(target_variable)
print("mask:")
print(mask)
print("max_target_len:", max_target_len)

input_variable:
tensor([[  36,    7,   51,  125,   35],
        [  28,   69,  116, 2075,   67],
        [ 393,   71,   77,    2,    2],
        [ 316, 4824, 1671,    0,    0],
        [ 328, 1608,    6,    0,    0],
        [ 337,    4,    2,    0,    0],
        [   4,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([8, 7, 6, 3, 3])
target_variable:
tensor([[   28,   148,   933,    51,    26],
        [  201,     4,     4,    38,   254],
        [  118,     4,  8213,   555,   118],
        [   24,     4, 11016,  1260,   267],
        [ 2335,     2,   596,     7,   173],
        [  337,     0,     4,     6,     7],
        [    4,     0,     2,     2,    97],
        [    2,     0,     0,     0,    77],
        [    0,     0,     0,     0,     4],
        [    0,     0,     0,     0,     2]])
mask:
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1],
      

In [0]:
class EncoderRNN(nn.Module):
  
  def __init__(self,hidden_size,embedding, n_layers=1, dropout=0):
    super(EncoderRNN,self).__init__()
    self.n_layers=n_layers
    self.hidden_size=hidden_size
    self.embedding=embedding
    self.gru=nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
  
  def forward(self, input_seq, input_lengths, hidden=None):
    embedded=self.embedding(input_seq)
    
    packed= torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
    outputs, hidden = self.gru(packed, hidden)
    
    outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
    
    outputs=outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
    
    return outputs, hidden

In [0]:
class Attn(torch.nn.Module):
  def __init__(self, method, hidden_size):
    super(Attn, self).__init__()
    self.method = method
    self.hidden_size = hidden_size
  
  def dot_score(self, hidden, encoder_output):
    return torch.sum(hidden * encoder_output, dim=2)
  
  def forward(self, hidden, encoder_outputs):
    attn_energies = self.dot_score(hidden, encoder_outputs) # (max_length, batch_size)
    # Transpose max_length and batch_size dims
    attn_energies = attn_energies.t() # (batch_size, max_length)
    # return the softmax normalized probability scores
    return F.softmax(attn_energies, dim=1).unsqueeze(1) # (batch_size, 1, max_length)

In [0]:
class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
    super(LuongAttnDecoderRNN, self).__init__()
    self.attn_model= attn_model
    self.hidden_size= hidden_size
    self.output_size= output_size
    self.n_layers = n_layers
    self.dropout = dropout
    
    self.embedding=embedding
    self.embedding_dropout = nn.Dropout(dropout)
    self.gru= nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers ==1 else dropout))
    self.concat= nn.Linear(hidden_size *2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    
    self.attn= Attn(attn_model, hidden_size)
    
  def forward(self, input_step, last_hidden, encoder_outputs):
    
    embedded = self.embedding(input_step)
    embedded = self.embedding_dropout(embedded)
    
    rnn_output, hidden = self.gru(embedded, last_hidden)
    
    attn_weights = self.attn(rnn_output, encoder_outputs)
    
    context= attn_weights.bmm(encoder_outputs.transpose(0,1))
    
    rnn_output = rnn_output.squeeze(0)
    context = context.squeeze(1)
    concat_input = torch.cat((rnn_output, context),1)
    concat_output = torch.tanh(self.concat(concat_input))
    
    output= self.out(concat_output)
    output= F.softmax(output, dim=1)
    
    return output, hidden

In [0]:
def maskNLLLoss(decoder_out, target, mask):
  nTotal = mask.sum()
  target = target.view(-1,1)
  
  gathered_tensor = torch.gather(decoder_out, 1, target)
  crossEntropy= -torch.log(gathered_tensor)
  
  loss=crossEntropy.masked_select(mask)
  
  loss= loss.mean()
  loss= loss.to(device)
  
  return loss, nTotal.item()

In [0]:
dec_o = torch.rand(5,7)
dec_o = F.softmax(dec_o, dim=1)
tar = torch.tensor([2,1,5,4,0], dtype=torch.long)
tar = tar.view(-1,1)
mask= torch.tensor([1,0,1,1,0], dtype=torch.uint8)
print(dec_o)
print(tar)
gath_ten= torch.gather(dec_o, 1, tar)


tensor([[0.0941, 0.1534, 0.1963, 0.0817, 0.1656, 0.1013, 0.2075],
        [0.1696, 0.1012, 0.1920, 0.1872, 0.0831, 0.0837, 0.1832],
        [0.2029, 0.1446, 0.1906, 0.1514, 0.1141, 0.1027, 0.0938],
        [0.1516, 0.1009, 0.1780, 0.1225, 0.1274, 0.1678, 0.1518],
        [0.1821, 0.1608, 0.0940, 0.1351, 0.1911, 0.1293, 0.1077]])
tensor([[2],
        [1],
        [5],
        [4],
        [0]])


In [0]:
small_batch_size=5
batches=batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input variable shape:", input_variable.shape)
print("lengths shape:", lengths.shape)
print("target_variable shape:", target_variable.shape)
print("mask shape", mask.shape)
print("max_target_len:", max_target_len)

hidden_size = 500
encoder_n_layers= 2
decoder_n_layers= 2
dropout = 0.1
attn_model = 'dot'
embedding= nn.Embedding(voc.num_words, hidden_size)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
encoder = encoder.to(device)
decoder = decoder.to(device)

encoder.train()
decoder.train()

encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.0001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.0001)
encoder_optimizer.zero_grad()
decoder_optimizer.zero_grad()

input_variable = input_variable.to(device)
lengths= lengths.to(device)
target_variable= target_variable.to(device)
mask=mask.to(device)

loss=0
print_losses=[]
n_totals=0

encoder_outputs, enceoder_hidden = encoder(input_variable, lengths)
print("Encoder outputs shape:", encoder_outputs.shape)
print("Last Encoder Hidden shape:", encoder_hidden.shape)

decoder_input= torch.LongTensor([[SOS_token for _ in range(small_batch_size)]])
decoder_input= decoder_input.to(device)
print("Initial decoder input shape:", decoder_input.shape)
print(decoder_input)

decoder_hidden = encoder_hidden[:decoder.n_layers]
print("Initial Decoder hidden state shape:", decoder_hidden.shape)
print("\n")
print("..............................................")
print("\n")


input variable shape: torch.Size([10, 5])
lengths shape: torch.Size([5])
target_variable shape: torch.Size([9, 5])
mask shape torch.Size([9, 5])
max_target_len: 9
Encoder outputs shape: torch.Size([10, 5, 500])


NameError: ignored

In [0]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [0]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [0]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [0]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [0]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [0]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 9.3379
Iteration: 2; Percent complete: 0.1%; Average loss: 9.1668
Iteration: 3; Percent complete: 0.1%; Average loss: 8.8485
Iteration: 4; Percent complete: 0.1%; Average loss: 8.4832
Iteration: 5; Percent complete: 0.1%; Average loss: 8.0054
Iteration: 6; Percent complete: 0.1%; Average loss: 7.5323
Iteration: 7; Percent complete: 0.2%; Average loss: 6.9689
Iteration: 8; Percent complete: 0.2%; Average loss: 6.8860
Iteration: 9; Percent complete: 0.2%; Average loss: 7.4703
Iteration: 10; Percent complete: 0.2%; Average loss: 7.2149
Iteration: 11; Percent complete: 0.3%; Average loss: 6.6770
Iteration: 12; Percent complete: 0.3%; Average loss: 6.2553
Iteration: 13; Percent complete: 0.3%; Average loss: 5.9191
Iteration: 14; Percent complete: 0.4%; Average loss: 5.7063
Iteration: 15; Percent complete: 0.4%; Average loss: 5.2586
Iteration: 16; Percent complete: 0.4%

In [0]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> what is your name?
Bot: berger .
> are you married?
Bot: no .
> how old are you?
Bot: i m not hungry .
> how are you?
Bot: okay .
> where are you from?
Bot: i m travelling .
> nasılsın?
Error: Encountered unknown word.
> are you stupid?
Bot: no .
> do you know me?
Bot: yes .
> who am i?
Bot: i don t know .
> what is your job?
Bot: i m not going to tell you .
> what is your problem?
Bot: i m not afraid of anything .
> are you robot?
Error: Encountered unknown word.
> what is my name?
Bot: berger .
> ai?
Error: Encountered unknown word.
> what do you want to me?
Bot: i m going to kill you .
> how do you kill me?
Bot: i told you .
> what is your plan?
Bot: i m not going to tell you .
> are you live?
Bot: yes .
> where?
Bot: the zoo .
> what is zoo?
Bot: the sheets . . .
> where is the zoo?
Bot: i don t know .
> are you animal?
Bot: yes .
> which animal?
Bot: the blond .
